In [5]:
from mltrainer import Trainer, TrainerSettings, ReportTypes, metrics
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor
import torch

import torch.optim as optim
optimizer = optim.Adam

import sys 
import os
sys.path.append(os.path.abspath('../models'))
sys.path.append(os.path.abspath('../dev'))

In [2]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()

batchsize = 32

streamers = fashionfactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()


2025-05-15 19:57:40.146 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/fashionmnist
2025-05-15 19:57:40.147 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/azureuser/.cache/mads_datasets/fashionmnist/fashionmnist.pt


In [ ]:
from pathlib import Path
from CNN import CNN

log_dir = Path("modellog").resolve()
if not log_dir.exists():
    log_dir.mkdir(parents=True)

accuracy = metrics.Accuracy()
loss_fn = torch.nn.CrossEntropyLoss()

model = CNN(units = 128)

settings = TrainerSettings(
    epochs=100,
    metrics=[accuracy],
    logdir=log_dir,
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
    earlystop_kwargs={
        "save": False,
        "verbose": True,
        "patience": 100,
    },
    scheduler_kwargs={
        "factor": 0.5,
        "patience": 10
    }
)
adam = optim.Adam(model.parameters(), lr=0.01)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optimizer,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    )

2025-05-15 19:57:43.797 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to /home/azureuser/machinelearning-melissa/notebooks/modellog/20250515-195743
2025-05-15 19:57:43.798 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


In [4]:
trainer.loop()

100%|██████████| 1875/1875 [00:18<00:00, 103.37it/s]
2025-05-15 19:58:05.381 | INFO     | mltrainer.trainer:report:205 - Epoch 0 train 0.4057 test 0.3302 metric ['0.8777']
100%|██████████| 1875/1875 [00:18<00:00, 101.23it/s]
2025-05-15 19:58:25.028 | INFO     | mltrainer.trainer:report:205 - Epoch 2 train 0.2933 test 0.3051 metric ['0.8888']
100%|██████████| 1875/1875 [00:18<00:00, 102.72it/s]
2025-05-15 19:58:44.331 | INFO     | mltrainer.trainer:report:205 - Epoch 4 train 0.2579 test 0.2958 metric ['0.8923']
100%|██████████| 1875/1875 [00:18<00:00, 101.44it/s]
2025-05-15 19:59:03.906 | INFO     | mltrainer.trainer:report:205 - Epoch 6 train 0.2365 test 0.2831 metric ['0.8996']
100%|██████████| 1875/1875 [00:18<00:00, 102.61it/s]
2025-05-15 19:59:23.261 | INFO     | mltrainer.trainer:report:205 - Epoch 8 train 0.2181 test 0.2793 metric ['0.9026']
100%|██████████| 1875/1875 [00:18<00:00, 102.68it/s]
2025-05-15 19:59:42.605 | INFO     | mltrainer.trainer:report:205 - Epoch 10 train 0.20